<a href="https://colab.research.google.com/github/grmurad/GM_SDS-CP022-ai-travel-companion/blob/main/Assisttravel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages and setup files for each time need to activate google colab session

!pip install transformers accelerate bitsandbytes
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install geonamescache
!pip install amadeus
!pip install aiohttp
!pip install flask flask-ngrok
!pip install pyngrok
!pip install fuzzywuzzy python-Levenshtein
!pip install googlemaps
import os
import shutil
from google.colab import drive

def setup_colab_environment():
    """
    Sets up the Colab environment by creating folders, mounting Google Drive,
    and copying necessary files.
    """

    # 1. Create folders
    os.makedirs('/content/templates', exist_ok=True)
    os.makedirs('/content/static', exist_ok=True)

    # 2. Mount Google Drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
    else:
        print("Google Drive is already mounted.")

    # 3. Define source and destination paths
    # ***VERIFY THIS PATH*** to match your Google Drive folder structure
    drive_base_path = '/content/drive/MyDrive/travel_app/Templates/Deep'
    colab_templates_path = '/content/templates/'
    colab_static_path = '/content/static/'
    colab_base_path = '/content/'


    # 4. Copy files
    # HTML files to templates
    html_files = ['index.html', 'results.html', 'selecionar_cidade.html']  # Replace with your HTML file names
    for file in html_files:
        #This is just for debugging
        print(os.path.join(drive_base_path, file))
        shutil.copy(os.path.join(drive_base_path, file), colab_templates_path)

    # CSV files to /content
    csv_files = ['cidades_filtradas.json', 'IATA_Cities.csv']  # Replace with your CSV file names
    for file in csv_files:
      #This is just for debugging
        print(os.path.join(colab_base_path, file))
        shutil.copy(os.path.join(drive_base_path, file), colab_base_path)

      # GIF file to static
    gif_file = 'HI9M.gif'  # Replace with your GIF file name
    shutil.copy(os.path.join(drive_base_path, gif_file), colab_static_path)

    print("Colab environment setup complete!")

# Call the function to set up the environment
setup_colab_environment()

In [2]:
from flask import Flask, request, render_template, redirect, url_for, Response
from flask_ngrok import run_with_ngrok
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import Accelerator
import torch
from amadeus import Client, ResponseError
from huggingface_hub import login
from google.colab import userdata
import re
import logging
import os
import asyncio
import aiohttp
from datetime import datetime, timedelta
import base64
from pyngrok import ngrok
from fuzzywuzzy import process
import json
import csv
import pandas as pd
import random
import googlemaps

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# Carregar a base de dados de cidades
with open("cidades_filtradas.json", "r", encoding="utf-8") as arquivo:
    BASE_CIDADES = json.load(arquivo)

def carregar_base_iata(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo, on_bad_lines='skip')
    required_columns = ["name", "iso_country", "municipality", "iata_code"]
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Coluna '{col}' não encontrada no arquivo CSV.")
    df = df.dropna(subset=['iata_code'])
    df = df[df['iata_code'] != '']
    df['municipality'] = df['municipality'].fillna('')
    base_iata = df[["name", "iso_country", "municipality", "iata_code"]].rename(columns={
        "name": "nome_aeroporto",
        "iso_country": "pais",
        "municipality": "cidade",
        "iata_code": "iata"
    }).to_dict(orient='records')
    return base_iata

BASE_IATA = carregar_base_iata("IATA_Cities.csv")

import random

# Lista de lugares famosos para turismo
LUGARES_FAMOSOS = [
    "Paris", "New York", "Tokyo", "Rome", "London",
    "Barcelona", "Dubai", "Sydney", "Rio de Janeiro", "Machu Picchu"
]

def buscar_imagem_aleatoria():
    lugar = random.choice(LUGARES_FAMOSOS)
    imagens = buscar_imagens_pixabay(lugar)
    return imagens[0] if imagens else None

    # Para requisições GET, buscar uma imagem aleatória
    imagem_fundo = buscar_imagem_aleatoria()
    return render_template("index.html", imagem_fundo=imagem_fundo)



# Initialize Google Maps client
gmaps = googlemaps.Client(key=userdata.get('Maps'))

def get_address_from_lat_lng(latitude, longitude):
    try:
        # Check if latitude and longitude are valid numerical values
        if latitude == "N/A" or longitude == "N/A":
            return "N/A"  # Return N/A if either value is missing

        # Reverse geocode if latitude and longitude are available
        reverse_geocode_result = gmaps.reverse_geocode((float(latitude), float(longitude)))
        if reverse_geocode_result:
            return reverse_geocode_result[0]["formatted_address"]
        return "N/A"
    except Exception as e:
        logger.error(f"Erro na API do Google Maps: {e}")
        return "N/A"

def batch_hotel_ids(hotel_ids, batch_size=50):
    """
    Split the list of hotel IDs into smaller batches.
    """
    for i in range(0, len(hotel_ids), batch_size):
        yield hotel_ids[i:i + batch_size]

def buscar_iata_por_cidade(nome_cidade, codigo_pais=None):
    nome_cidade = nome_cidade.strip().lower()
    resultados = []
    for cidade in BASE_IATA:
        cidade_nome = str(cidade.get("cidade", "")).lower()
        if nome_cidade in cidade_nome:
            if codigo_pais and cidade.get("pais", "").lower() != codigo_pais.lower():
                continue
            resultados.append({
                "nome": cidade["cidade"],
                "iata": cidade["iata"]
            })
    return resultados

import requests

PIXABAY_API_KEY = userdata.get('Pixabay')


def buscar_imagens_pixabay(query):
    url = f"https://pixabay.com/api/?key={PIXABAY_API_KEY}&q={query}&image_type=photo&per_page=5"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return [imagem["largeImageURL"] for imagem in data.get("hits", [])]
    return []

ngrok.set_auth_token(userdata.get('NGROK'))
app = Flask(__name__)

# Middleware to add custom User-Agent header
@app.after_request
def add_custom_user_agent(response):
    response.headers['User-Agent'] = 'Custom-Agent'
    return response

public_url = ngrok.connect(5000).public_url

login(token=userdata.get('HUGGINGFACE_TOKEN'))
AMADEUS_CLIENT_ID = userdata.get('AMADEUS_API_KEY')
AMADEUS_CLIENT_SECRET = userdata.get('AMADEUS_API_SECRET')

if not AMADEUS_CLIENT_ID or not AMADEUS_CLIENT_SECRET:
    raise ValueError("Amadeus credentials not found in Colab Secrets. Please add them.")

amadeus = Client(
    client_id=AMADEUS_CLIENT_ID,
    client_secret=AMADEUS_CLIENT_SECRET)

# Restante do código Flask (modelo, funções, rotas, etc.) permanece o mesmo...

async def get_session():
    return aiohttp.ClientSession()

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
accelerator = Accelerator()
model, tokenizer = accelerator.prepare(model, tokenizer)

def validate_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def extrair_cidades_e_iata(texto):
    padrao = r"([A-Za-zÀ-ÿ,\s]+)\s*\(([A-Z]{3})\)"
    matches = re.findall(padrao, texto)
    cidades = [{"nome": cidade.title().strip(), "iata": iata.strip()} for cidade, iata in matches]
    return cidades

def buscar_cidades_similares_local(nome_cidade, limite=5):
    nomes_cidades = [cidade["nome"] for cidade in BASE_CIDADES]
    resultados = process.extract(nome_cidade, nomes_cidades, limit=limite)
    cidades_similares = []
    for nome, score in resultados:
        if score >= 70:
            cidade = next((c for c in BASE_CIDADES if c["nome"] == nome), None)
            if cidade:
                cidades_similares.append(cidade)
    return cidades_similares

async def buscar_hoteis_por_cidade(session, cidade_iata, access_token):
    try:
        async with session.get(
            "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city",
            params={"cityCode": cidade_iata},
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                hotel_data = (await response.json()).get("data", [])
                if not hotel_data:
                    return "No hotels found for this city."

                # Extract hotel IDs and names
                hotel_list = [{"id": hotel.get("hotelId", "N/A"), "name": hotel.get("name", "N/A")} for hotel in hotel_data]
                print(hotel_list)
                return hotel_list
            else:
                error_response = await response.text()
                logger.error(f"Failed to fetch hotels: {response.status}, Response: {error_response}")
                return None
    except Exception as e:
        logger.error(f" Buscar_hoteis: Erro na API Amadeus: {e}")
        return None


async def buscar_ofertas_multiplos_hoteis(session, hotel_ids, access_token, check_in_date, check_out_date):
    try:
        detailed_hotels = []

        # Split hotel IDs into batches
        for batch in batch_hotel_ids(hotel_ids):
            params = {
                "hotelIds": ",".join(batch),  # Convert batch of hotel IDs to a comma-separated string
                "checkInDate": check_in_date,
                "checkOutDate": check_out_date,
                "roomQuantity": 1,  # Number of rooms
                "adults": 1,  # Number of adults
                "currency": "USD",  # Currency for pricing
                "bestRateOnly": "false",  # Allow multiple offers per hotel
                "includeClosed": "false",  # Exclude closed hotels
                "lang": "EN"  # Language for responses
            }

            async with session.get(
                "https://test.api.amadeus.com/v3/shopping/hotel-offers",
                params=params,
                headers={"Authorization": f"Bearer {access_token}"}
            ) as response:
                if response.status == 200:
                    hotel_data = (await response.json()).get("data", [])
                    if not hotel_data:
                        continue

                    # Extract details for each hotel
                    for hotel in hotel_data:
                        hotel_name = hotel.get("hotel", {}).get("name", "N/A")
                        hotel_id = hotel.get("hotel", {}).get("hotelId", "N/A")
                        latitude = hotel.get("hotel", {}).get("latitude", "N/A")
                        longitude = hotel.get("hotel", {}).get("longitude", "N/A")
                        offers = hotel.get("offers", [])

                        # Get full address using Google Maps Geocoding API
                        endereco = get_address_from_lat_lng(latitude, longitude)

                        # Extract price details (if available)
                        preco = "N/A"
                        if offers:
                            preco = offers[0].get("price", {}).get("total", "N/A")

                        hotel_info = {
                            "nome": hotel_name,
                            "id": hotel_id,
                            "endereco": endereco,
                            "preco": preco
                        }
                        detailed_hotels.append(hotel_info)
                else:
                    error_response = await response.text()
                    logger.error(f"Failed to fetch hotel offers: {response.status}, Response: {error_response}")
                    continue

        return detailed_hotels
    except Exception as e:
        logger.error(f"Erro na API Amadeus: {e}")
        return None

async def buscar_detalhes_hotel(session, hotel_id, access_token):
    try:
        async with session.get(
            f"https://test.api.amadeus.com/v1/reference-data/locations/hotels/{hotel_id}",
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                hotel_details = await response.json()
                return hotel_details.get("data", {})
            else:
                logger.error(f"Failed to fetch hotel details: {response.status}")
                return None
    except Exception as e:
        logger.error(f"Buscar Detalhes Hotel: Erro na API Amadeus: {e}")
        return None

def get_default_departure_date():
    today = datetime.today()
    default_date = today + timedelta(days=10)
    return default_date.strftime("%Y-%m-%d")

import unicodedata

def clean_city_name(city_name):
    """Removes accents and special characters from a city name."""
    # Normalize to NFKD form to decompose accented characters
    city_name = unicodedata.normalize('NFKD', city_name)

    # Remove accents and keep only ASCII characters
    city_name = ''.join(c for c in city_name if not unicodedata.combining(c))

    # Convert to lowercase for consistency
    city_name = city_name.encode('ASCII', 'ignore').decode('ASCII').lower()

    return city_name

def gerar_sugestoes_viagem(prompt_usuario, num_sugestoes=3):
    prompt_usuario += (
        f" Liste {num_sugestoes} de destinos turísticos, incluindo:"
        f" - Informe cidades com serviço regular de voos que sejam proximas às atracoes destacadas: Nome da Cidade e seu codigo IATA em parentesis: (exemplo: 'San Juan (SJU)')."
        f" - Principais atrações e atividades. use no maximo 1000 palavras de forma que resuma bem o pedido do usuario"
    )
    inputs = tokenizer(prompt_usuario, return_tensors="pt").to("cuda")
    inputs["attention_mask"] = torch.ones_like(inputs["input_ids"]).to("cuda")
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=1000,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True).strip()
    print(response)
    response = response.replace(prompt_usuario, "").strip()
    return response, extrair_cidades_e_iata(response)

async def buscar_hoteis(session, cidade_iata, access_token):
    try:
        async with session.get(
            "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city",
            params={"cityCode": cidade_iata},
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                hotel_data = (await response.json()).get("data", [])
                if not hotel_data:
                    return "No hotels found for this city."

                hotel_list = []
                for hotel in hotel_data[:5]:  # Limita a 5 hotéis
                    hotel_id = hotel.get("hotelId")
                    hotel_name = hotel.get("name", "N/A")

                    # Buscar detalhes do hotel
                    hotel_details = await buscar_detalhes_hotel(session, hotel_id, access_token)
                    if hotel_details:
                        endereco = hotel_details.get("address", {}).get("lines", ["N/A"])[0]
                        telefone = hotel_details.get("contact", {}).get("phone", "N/A")
                        preco = hotel_details.get("price", {}).get("amount", "N/A")
                    else:
                        endereco = "N/A"
                        telefone = "N/A"
                        preco = "N/A"

                    hotel_info = {
                        "nome": hotel_name,
                        "id": hotel_id,
                        "endereco": endereco,
                        "telefone": telefone,
                        "preco": preco
                    }
                    hotel_list.append(hotel_info)

                return hotel_list
            else:
                logger.error(f"Failed to fetch hotels: {response.status}")
                return None
    except Exception as e:
        logger.error(f"Buscar_Hoteis: Erro na API Amadeus: {e}")
        return None

async def buscar_voos(session, origem_iata, destino, access_token, data_partida="2025-03-10", adultos=1):
    try:
        params = {
            "originLocationCode": origem_iata,
            "destinationLocationCode": destino,
            "departureDate": data_partida,
            "adults": adultos,
            "max": 5  # Limit to 5 results
        }
        async with session.get(
            "https://test.api.amadeus.com/v2/shopping/flight-offers",
            params=params,
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                flight_data = (await response.json()).get("data", [])
                if not flight_data:
                    logger.warning(f"No flight data found for {origem_iata} -> {destino}")
                    return None  # Retorna None se não houver voos

                # Fetch airline names
                airline_codes = set()
                for flight in flight_data:
                    for segment in flight["itineraries"][0]["segments"]:
                        airline_codes.add(segment["carrierCode"])

                # Map carrier codes to airline names
                airline_names = {}
                for code in airline_codes:
                    async with session.get(
                        "https://test.api.amadeus.com/v1/reference-data/airlines",
                        params={"airlineCodes": code},
                        headers={"Authorization": f"Bearer {access_token}"}
                    ) as airline_response:
                        if airline_response.status == 200:
                            airline_info = (await airline_response.json()).get("data", [])
                            if airline_info:
                                airline_names[code] = airline_info[0].get("businessName", code)

                # Format the flight list with more details
                flights = []
                for flight in flight_data:
                    segments = flight["itineraries"][0]["segments"]
                    flight_info = {
                        "airline": airline_names.get(segments[0].get("carrierCode", "N/A"), "N/A"),
                        "flightNumber": segments[0].get("number", "N/A"),
                        "origin": segments[0].get("departure", {}).get("iataCode", "N/A"),
                        "destination": segments[-1].get("arrival", {}).get("iataCode", "N/A"),
                        "departureTime": segments[0].get("departure", {}).get("at", "N/A"),
                        "arrivalTime": segments[-1].get("arrival", {}).get("at", "N/A"),
                        "price": flight["price"].get("total", "N/A"),
                        "currency": flight["price"].get("currency", "N/A")
                    }
                    flights.append(flight_info)
                return flights
            else:
                error_response = await response.text()
                logger.error(f"Failed to fetch flights: {response.status}, Response: {error_response}")
                return None  # Retorna None em caso de erro
    except Exception as e:
        logger.error(f"Buscar_Voos : Erro na API Amadeus: {e}")
        return None  # Retorna None em caso de exceção

async def get_amadeus_token(session):
    try:
        credentials = base64.b64encode(
            f"{AMADEUS_CLIENT_ID}:{AMADEUS_CLIENT_SECRET}".encode("utf-8")
        ).decode("utf-8")
        async with session.post(
            "https://test.api.amadeus.com/v1/security/oauth2/token",
            headers={
                "Content-Type": "application/x-www-form-urlencoded",
                "Authorization": f"Basic {credentials}"
            },
            data={"grant_type": "client_credentials"}
        ) as response:
            if response.status == 200:
                token_data = await response.json()
                return token_data.get("access_token")
            else:
                error_response = await response.text()
                logger.error(f"Failed to fetch access token: {response.status}, Response: {error_response}")
                return None
    except Exception as e:
        logger.error(f"Error fetching access token: {e}")
        return None

async def fetch_hotels_and_flights(origem_iata, cidades, data_partida):
    print("origem IATA:", origem_iata)
    session = await get_session()
    access_token = await get_amadeus_token(session)
    results = []

    # Calculate check-in and check-out dates (e.g., 3-day stay)
    check_in_date = data_partida
    check_out_date = (datetime.strptime(data_partida, "%Y-%m-%d") + timedelta(days=3)).strftime("%Y-%m-%d")

    for cidade in cidades:
        cidade_iata = cidade["iata"]
        cidade_nome = cidade["nome"]

        # Step 1: Fetch hotel list by city
        logger.debug(f"Fetching hotels for city: {origem_iata} {cidade_iata}")
        hotel_list = await buscar_hoteis_por_cidade(session, cidade_iata, access_token)
        logger.debug(f"Hotel list for {origem_iata} {cidade_iata}: {hotel_list}")

        if not hotel_list:
            results.append({
                "city": cidade["nome"],
                "iata": cidade_iata,
                "hotels": "Nenhum hotel encontrado.",
                "flights": None
            })
            continue

        # Step 2: Fetch hotel offers for all hotels in the city (in batches)
        hotel_ids = [hotel["id"] for hotel in hotel_list]
        logger.debug(f"Fetching hotel offers for hotel IDs: {hotel_ids}")
        detailed_hotels = await buscar_ofertas_multiplos_hoteis(session, hotel_ids, access_token, check_in_date, check_out_date)
        logger.debug(f"Detailed hotels for {cidade_iata}: {detailed_hotels}")

        # Step 3: Filter the 5 hotels with the lowest fares
        if detailed_hotels:
            # Remove hotels with no price data
            detailed_hotels_with_price = [hotel for hotel in detailed_hotels if hotel["preco"] != "N/A"]
            # Sort by price (ascending)
            sorted_hotels = sorted(detailed_hotels_with_price, key=lambda x: float(x["preco"]))
            # Select the top 5
            top_5_hotels = sorted_hotels[:5]
        else:
            top_5_hotels = []

        # Step 4: Fetch flights
        logger.debug(f"Fetching flights for {origem_iata} -> {cidade_iata}")
        flights = await buscar_voos(session, origem_iata, cidade_iata, access_token, data_partida)
        logger.debug(f"Flights for {cidade_iata}: {flights}")

        results.append({
            "city": cidade_nome,
            "iata": cidade_iata,
            "hotels": top_5_hotels if top_5_hotels else "Nenhum hotel encontrado.",
            "flights": flights if flights else "Nenhum voo encontrado."
        })

    await session.close()
    logger.debug(f"Final results: {results}")
    return results

@app.route("/selecionar_cidade", methods=["POST"])
def selecionar_cidade():
    origem = request.form.get("origin")
    prompt_usuario = request.form.get("preferences")
    data_partida = request.form.get("date")
    selected_city_iata = request.form.get("selected_city")

    logger.info(f"Selected IATA code: {selected_city_iata}")
    logger.info(f"Origem: {origem}")
    logger.info(f"Prompt do usuário: {prompt_usuario}")
    logger.info(f"Data de partida: {data_partida}")

    if selected_city_iata:
        origem_iata = selected_city_iata
    else:
        resultados = buscar_iata_por_cidade(origem)
        if not resultados:
            return render_template("index.html", error=f"Não foi possível encontrar o código IATA para: {origem}", preferences=prompt_usuario, origin=origem, date=data_partida)
        if len(resultados) > 1:
            return render_template("selecionar_cidade.html", cidades=resultados, origem=origem, prompt_usuario=prompt_usuario, date=data_partida)
        origem_iata = resultados[0]["iata"]
        print("origem IATA: em selecionar_cidade", origem_iata)
        print("lista cidades em selecionar_cidades", cidades)

    if not data_partida:
        data_partida = get_default_departure_date()
        logger.info(f"Nenhuma data de partida fornecida. Usando data padrão: {data_partida}")
    elif not validate_date(data_partida):
        return render_template("index.html", error="Formato de data inválido. Use AAAA-MM-DD.", preferences=prompt_usuario, origin=origem, date=data_partida)

    sugestoes, cidades = gerar_sugestoes_viagem(prompt_usuario)
    logger.info(f"Sugestões geradas: {sugestoes}")
    logger.info(f"Cidades encontradas: {cidades}")

    results = asyncio.run(fetch_hotels_and_flights(origem_iata, cidades, data_partida))
    logger.info(f"Resultados da busca: {results}")

    # Buscar imagens para cada cidade
    imagens = {}
    for cidade in cidades:
        query = cidade["nome"]
        imagens[query] = buscar_imagens_pixabay(query)
#    print("Imagens encontradas:", imagens)  # Depuração
    return render_template("results.html", suggestions=sugestoes, results=results, imagens=imagens)


# Lista de lugares famosos para turismo
LUGARES_FAMOSOS = [
    "Paris", "New York", "Tokyo", "Rome", "London", "San Francisco"
    "Barcelona", "Dubai", "Sydney", "Rio de Janeiro", "Machu Picchu",
    "Madrid", "Viena", "Africa", "Lisboa", "Los Angeles", "Cancun"
]

def buscar_imagem_aleatoria():
    lugar = random.choice(LUGARES_FAMOSOS)
    imagens = buscar_imagens_pixabay(lugar)
    return imagens[0] if imagens else None

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        origem = request.form.get("origin")
        origem = clean_city_name(origem)
        prompt_usuario = request.form.get("preferences")
        data_partida = request.form.get("date")

        if not origem:
            return render_template("index.html", error="Por favor, forneça a cidade de origem.")

        resultados = buscar_iata_por_cidade(origem)
        if not resultados:
            return render_template("index.html", error=f"Nenhuma cidade encontrada para: {origem}. Procure não usar acentuação ou ~ !", preferences=prompt_usuario, origin=origem, date=data_partida)

        if len(resultados) > 1:
            return render_template("selecionar_cidade.html", cidades=resultados, origem=origem, prompt_usuario=prompt_usuario, data_partida=data_partida)

        origem_iata = resultados[0]["iata"]

        if not data_partida:
            data_partida = get_default_departure_date()
        elif not validate_date(data_partida):
            return render_template("index.html", error="Formato de data inválido. Use AAAA-MM-DD.", preferences=prompt_usuario, origin=origem, date=data_partida)

        sugestoes, cidades = gerar_sugestoes_viagem(prompt_usuario)
        results = asyncio.run(fetch_hotels_and_flights(origem_iata, cidades, data_partida))

        # Buscar imagens para cada cidade
        imagens = {}
        for cidade in cidades:
            query = cidade["nome"]
            imagens[query] = buscar_imagens_pixabay(query)

        return render_template("results.html", suggestions=sugestoes, results=results, imagens=imagens)

    # Para requisições GET, buscar uma imagem aleatória
    imagem_fundo = buscar_imagem_aleatoria()
    return render_template("index.html", imagem_fundo=imagem_fundo)
if __name__ == "__main__":
    print(f" * Running on {public_url}")
    app.run()

Mon Mar 17 00:37:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 00:42:25] "POST /selecionar_cidade HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 00:45:25] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 00:45:25] "GET /static/favicon.ico HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 00:45:25] "GET /static/favicon.ico HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 00:46:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 00:47:35] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 00:47:35] "GET /static/HI9M.gif HTTP/1.1" 200 -


Quero conhecer culturas orientais no oeste da asia Liste 3 de destinos turísticos, incluindo: - Informe cidades com serviço regular de voos que sejam proximas às atracoes destacadas: Nome da Cidade e seu codigo IATA em parentesis: (exemplo: 'San Juan (SJU)'). - Principais atrações e atividades. use no maximo 1000 palavras de forma que resuma bem o pedido do usuario.

1. Hong Kong (HKG)
- Attractions: The Peak, Victoria Harbour, Temple Street Night Market, Big Buddha (Tian Tan Buddha), Ocean Park Hong Kong, and the Avenue of Stars.
- Activities: Enjoy the stunning views of the city from The Peak, take a ferry ride across Victoria Harbour, haggle for souvenirs at the Temple Street Night Market, visit the Big Buddha for spiritual enlightenment, experience thrilling rides at Ocean Park, and pay tribute to local celebrities at the Avenue of Stars.

2. Tokyo (NRT/HND)
- Attractions: Tokyo Skytree, Shibuya Crossing, Meiji Shrine, Senso-ji Temple, Akihabara, and Robot Restaurant.
- Activities:

ERROR:__main__:Failed to fetch hotel offers: 400, Response: {"errors":[{"code":3289,"title":"RATE NOT AVAILABLE FOR REQUESTED DATES","status":400,"detail":"Provider Error - RATE NOT AVAILABLE FOR REQUESTED DATES","source":{"parameter":"hotelIds=YXHKGDHH"}},{"code":1257,"title":"INVALID PROPERTY CODE","status":400,"detail":"Provider Error - INVALID PROPERTY CODE","source":{"parameter":"hotelIds=YXHKG001,YXHKG296,YXHKG700,YXHKG707,YXHKG993,YXHKGCOS,YXHKGDOR,YXHKGDRH,YXHKGLHO,YXHKGODF,YXHKGVWA"}}]}


[{'id': 'ALBKK211', 'name': 'ALOFT BANGKOK SUKHUMVIT'}, {'id': 'AZBKK512', 'name': 'SOMERSET SUKHUMVIT THONGLOR'}, {'id': 'AZBKK128', 'name': 'SOMERSET PARK SUANPLU'}, {'id': 'AZBKK127', 'name': 'SOMERSET LAKE POINT'}, {'id': 'BWBKK353', 'name': 'BW PLUS AT 20 SUKHUMVIT'}, {'id': 'BWBKK330', 'name': 'BW PREMIER AMARANTH SUVARNAB'}, {'id': 'BWBKK328', 'name': 'BEST WESTERN MAYFAIR SUITES'}, {'id': 'BWBKK356', 'name': 'BEST WESTERN PREMIER SUKHUMVIT'}, {'id': 'BYBKK800', 'name': 'BANYAN TREE BANGKOK'}, {'id': 'CHBKKBSR', 'name': 'BS PREMIER AIRPORT HOTEL'}, {'id': 'CNBKK260', 'name': 'CONRAD BANGKOK RESIDENCES'}, {'id': 'CPBKK3C3', 'name': 'CROWNE PLAZA LUMPINI PARK'}, {'id': 'CYBKKCYC', 'name': 'COURTYARD BY MARRIOTT BANGKOK'}, {'id': 'DHBKKAVB', 'name': 'MODE HOTEL SATHORN'}, {'id': 'DHBKKAUU', 'name': 'SM GRANDE RESIDENCE'}, {'id': 'DHBKKAWB', 'name': 'HOLIDAY INN EXPRESS BANGKOK SATHORN'}, {'id': 'FGBKKMHB', 'name': 'MONTIEN HOTEL BANGKOK'}, {'id': 'FGBKKBEL', 'name': 'BELAIRE BANGKO

ERROR:__main__:Failed to fetch hotel offers: 400, Response: {"errors":[{"code":3289,"title":"RATE NOT AVAILABLE FOR REQUESTED DATES","status":400,"detail":"Provider Error - RATE NOT AVAILABLE FOR REQUESTED DATES","source":{"parameter":"hotelIds=YRSINRAF"}},{"code":1257,"title":"INVALID PROPERTY CODE","status":400,"detail":"Provider Error - INVALID PROPERTY CODE","source":{"parameter":"hotelIds=YXSIN336,YXSIN411,YXSIN554,YXSINDOR,YXSINMBS,YXSINMER"}}]}
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 00:49:25] "POST /selecionar_cidade HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Mar/2025 00:50:12] "GET / HTTP/1.1" 200 -
